In [4]:

# for sentence play in Anki
# initial date: 2015-11-20

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import re

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\mae\\sentencePlayAnki.txt', 'w', 'utf-8')

# read list of words with images [captions] into imglist
with codecs.open('E:\\1Now\\taglist\\mwlDailyImg20141124_20151205.txt', 'r', 'utf-8') as f:
    imglist = f.read().splitlines() # strip '\n'
#     f.close() # not necessary with with statement
imginhw = [] # track which images match head words

# tag word lists
tfiles = ['acaWriting.txt', 'airTravel.txt', 'bodyPart.txt', 'campus.txt', 'economics.txt', 'emotion.txt', 
          'employment.txt', 'environment.txt', 'family.txt', 'health.txt', 'job.txt', 'legal.txt', 
          'nonCount.txt', 'personality.txt', 'sports.txt', 'timeAdv.txt', 'timePeriod.txt', 'weather.txt',
          'core31only.txt', 'tg.txt']
topics = []
for tfile in tfiles:
    topic = {}
    with codecs.open('E:\\1Now\\taglist\\' + tfile, 'r', 'utf-8') as f:
        twords = f.read().splitlines()
        f.close()
    topic['words'] = twords
    topic['topic'] = tfile.split('.')[0]
    topics.append(topic)
    

# read TOEFL/GRE word list
# with codecs.open('E:\\1Now\\mae\\tginmwaled.txt', 'r', 'utf-8') as f:
#     tgwl = f.read().splitlines() # strip '\n'
#     f.close()

#results = geweiDict.find({"fr":"y"}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, 
#                              "idpvs":1, "sublist":1, "audiofile":1, "alles":1})
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, "img":1,
                              "idpvs":1, "sublist":1, "homog":1, "alles":1, "tag":1, "label":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)

# total number of head words
totalhw = 0

# threshold of the length of definition
defthres = 12

# Produce Anki note fields and tags:
for result in results:
    
    word = result.get("hw")    
    if word:
        #print word
        totalhw += 1
    else:
        print "no head word"
        continue

    
    # get items of result
    tagindb = result.get('tag')
    label = result.get('label')
    
    infls = result.get("infl")
    subhws = result.get("subhw")   
    imgindb = result.get("img")
    
#     homograph = 0
    homog = result.get("homog")
    
    idpvs = result.get("idpvs")
    sublists = result.get("sublist") #sublists is a list whose items are dictionary
    
#     fr = result.get("fr")

#     alles = result.get("alles")        
#     if alles: totales = len(alles)
#     else: totales=0
        
#     if totales >= 100: nes = "100+"
#     elif totales >= 50: nes = "50-99"
#     elif totales >= 10: nes = "10-49"
#     elif totales >=  3: nes = "3-9"
#     elif totales >=  2: nes = "2"
#     elif totales >=  1: nes = "1"
#     else: nes = "0"
        
    
    if sublists:
        cnt = 1
        idiom = 0
        
        # each "es" or "idpv" of sublist in sublists will be an Anki note
        for sublist in sublists: #sublist is dictionary        
            
            # 9 fields with tags: PlaySent,def,Wow,prn,hw,audio,footnote,img,caption   tags
            
            # Anki note field #1: PlaySent (incomplete example sentences for cloze test)
            # PlaySent must be the first field because codes for it include continue statement which
            # may not create a note for this loop.
            if sublist.has_key("es"):                
                # no example sentence but have definition, write head word to PlaySent field
                if len(sublist["es"]) == 0 and len(sublist["defi"]) >= defthres:
                    #print defthres, " --- ", word                    
                    fdict.write(word)
                # this es is empty and "def" is not a definition, discard this fake definition
                elif len(sublist["es"]) == 0 and len(sublist["defi"]) < defthres:
                    #print word, ": empyt example and not a defintion"
                    continue                    
                else:
                    for sPlay in sublist["es"]:                    
                        #sPlay = es.encode('ascii', 'ignore').replace(word, "___")                        
                        #sPlay = es
                        insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                        sPlay = insensitive_replace.sub('___', sPlay)
                        
                        if infls:
                            for infl in infls:
                                sPlay = sPlay.replace(infl, "...")
                        if subhws:
                            for subhw in subhws:
                                sPlay = sPlay.replace(subhw, "???")
                        fdict.write(str(cnt) + '<font color="blue">-es: </font>' + sPlay + "<br>")
                    
            if sublist.has_key("idpv"):
                if len(sublist["idpv"]) == 0:
                    #print word, ": empyt idpv"
                    continue
                #sPlay = sublist["idpv"].encode('ascii', 'ignore').replace(word, "___")
                #sPlay = sublist["idpv"].encode('ascii', 'ignore')
                sPlay = sublist["idpv"]
                sPlay = re.sub(r'\s+(?=\d)', '<br><br>', sPlay) # replace the space before a number with '<br><br>'
                
                insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                sPlay = insensitive_replace.sub('___', sPlay)
                if sPlay.find(":") >= 0: idiom = 1
                
                if infls:
                    for infl in infls:
                        sPlay = sPlay.replace(infl, "...")
                if subhws:
                    for subhw in subhws:
                        sPlay = sPlay.replace(subhw, "???")
                fdict.write('<font color="blue">*idpv*</font> ' + sPlay + "<br>")
            #if cnt%1 == 0: fdict.write('<br>')
            fdict.write(" \t")               
            

            # Anki note field: definition
            if sublist.has_key("defi"):                
                #fdict.write(str(cnt) + '-def: ' + sublist["def"].encode('ascii', 'ignore') + "<br><br>")
                fdict.write(str(cnt) + '-def: ' + sublist["defi"] + "<br><br>")            
            fdict.write(" \t")
                
            # Anki note field: Wow (complete example sentences)
            if sublist.has_key("es"):  
                for es in sublist["es"]:
                    fdict.write(str(cnt) + '<font color="red">-es: </font>' + 
                                es + "<br>")                
                                #es.encode('ascii', 'ignore') + "<br>")                
            if sublist.has_key("idpv"):                
                fdict.write('<font color="red">*idpv*</font> ' + 
                            re.sub(r'\s+(?=\d)', '<br><br>', sublist["idpv"]) + "<br>")
                            #sublist["idpv"].encode('ascii', 'ignore') + "<br>")
            #if cnt%1 == 0: fdict.write('<br>') 
            fdict.write(" \t")
            
       
            
            # Anki note field: Pronunciation in ASCII IPA symbols
            prns = result.get("prn")
            if prns:
                for prn in prns:
                    fdict.write("/" + prn + "/<br>")
            fdict.write(" \t")  # add one space before tab to separate tabs in case a field is empty.     
            
            # Anki note field: head word
            fdict.write(word + "\t")     
            
            
            # Anki note field: Audio    
            if homog:
                fdict.write("[sound:" + homog + ".wav]")
#                 homograph = 1                
            elif word == 'con':                    
                # con is a key word of Windows, a filename as 'con.wav' is not allowed, so use 'con1.wav'                
                print "---------con-----------"
                fdict.write("[sound:con1.wav]")                   
            else: 
                fdict.write("[sound:" + word + ".wav]")
                    
            if subhws:
                for subhw in subhws:
                    #fdict.write("<br>[sound:" + subhw.encode('ascii', 'ignore') + ".wav]")
                    fdict.write("<br>[sound:" + subhw + ".wav]")
            fdict.write(" \t")    

            
            # make tags 
            tags = ''
            if tagindb:
                for tag in tagindb:
                    tags += ' ' + tag
            for topic in topics:
                if word in topic['words']: tags += ' ' + topic['topic']
                    
            if label:
                if 'slang' in label: tags += ' slang'
                if 'baseball' in label: tags += ' baseball'
                if 'sports' in label: tags += ' sports+' # beyond core sports.txt
                
            if idiom == 1: tags += " idiom"
#             if fr: tags += " fr" # Foreign word (French, Mexican Spanish)       
#             if homograph == 1 and word != 'con' : tags += " homog" # homog -- homograph
#             if word in tgwl: tags += ' tg'
#             tags += " es" + nes # level of a word by the number of examples
            
            
            # Anki note field: Footnote            
            fdict.write("<br><br>{" + tags + "}<br><br>\t")
            
            # Anki note fields: Image and Caption
            img = ' '
            caption = ' '
            imgflag = 0
            imgcnt = 0           
            # check if hw has images in db (mwl book)
            if imgindb:
                for x in imgindb:
                    img += '<img src="' + x + '" />'
                
            # check if hw has images in mwl daily images
            for imgname in imglist:
                if word == imgname.split('.')[0]: 
                    imginhw.append(imgcnt) # record the index of items in imglist that match hw
                    imgl = imgname.split(';')
                    #print imgl
                    img += '<img src="' + imgl[0] + '" />'
                    imgflag = 1
                    if len(imgl) == 2: caption += imgl[1]
                    elif len(imgl) == 3: caption += imgl[1] + ';' + imgl[2]
                
                imgcnt += 1
                 
            fdict.write(img + "\t")
            fdict.write(caption + "\t")

            # Anki tags
            if imgflag == 1: tags += " imgmwlD" # images from mwl daily images
            fdict.write(tags + "\n")
            
            cnt += 1
    
fdict.close()
print "total head words: ", totalhw


# fd = codecs.open('E:\\1Now\\mae\\imgNameInHw.txt', 'w', 'utf-8')
# for i in range(0, len(imglist)):
#     if i not in imginhw: print imglist[i]
#     else: fd.write(imglist[i] + '\n')
# fd.close()  
    




---------con-----------
---------con-----------
---------con-----------
---------con-----------
---------con-----------
---------con-----------
total head words:  39195


In [14]:
# create an Anki import file with only hw not empty for a word list
# 
import codecs

fout = codecs.open('E:\\1Now\\mae\\3fields1tag4sPlayAnki.txt', 'w', 'utf-8')

with codecs.open('E:\\1Now\\mae\\tgnotinmwaled.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()
cnt = 0
for line in lines:
    if not line: continue
    # 9 fields with tags: PlaySent,def,Wow,prn,hw,audio,footnote,img,caption   tags
    # first field must not be empty, so use hw to hold the position
    # PlaySent, hw, audio, tags not empty
    anote = line + '\t \t \t \t' + line+'\t' + '[sound:'+line+'.wav]\t' + ' \t \t \tnotinmwaled tg'
    #print anote
    fout.write(anote + '\n')
    cnt += 1
print cnt, ' words'    
fout.close()    
    

164  words


In [5]:
a = [1,2,3]
for i in range(0, len(a)):
    print a[i]

1
2
3


In [2]:
# this snippets of codes organize images list to a unique sorted list
# first add new items in imglistraw.txt, the snippets will create imglist.txt which will be used in codes above.

import codecs

fimg = codecs.open('E:\\1Now\\mae\\4Anki_imgNameCaption.txt', 'w', 'utf-8')

with codecs.open('E:\\1Now\\mae\\4Anki_imgNameCaptionraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()

cnt = 0    
# unique, insensitive case sort
uImgs = sorted(set(lines), key=lambda s: s.lower())

for img in uImgs:  
    fimg.write(img + '\n')
    cnt += 1
    #print cnt, ': ', img
print cnt
fimg.close()


708


In [11]:
a = [1, 2, 3, 4, 5]
a[1:]

[2, 3, 4, 5]

In [6]:
word="abc\nu0x99999"
print word
print word.encode('ascii', 'ignore')

abc
u0x99999
abc
u0x99999


In [2]:
tstr="we adjust oursleves to the weather"
print tstr.replace("adjust", "___")

we ___ oursleves to the weather


In [25]:
sPlay = "You mean he lied to us? Bingo!"
repl = "bingo"
resub = re.compile(re.escape(repl), re.IGNORECASE)
print resub.sub("___", sPlay) # sPlay doesn't change
print sPlay

You mean he lied to us? ___!
You mean he lied to us? Bingo!


In [5]:
with codecs.open('anki\\imagelist.txt', 'r', 'utf-8') as f:
    imglist1 = f.read().splitlines() # strip '\n'
with codecs.open('anki\\imagelist.txt', 'r', 'utf-8') as f:    
    imglist2 = f.readlines() # include '\n'

print imglist1[0:5]
print imglist2[0:5]


[u'aardvark.gif', u'abacus.gif', u'abscissa.gif', u'accordion.gif', u'airplane.gif']
[u'aardvark.gif\n', u'abacus.gif\n', u'abscissa.gif\n', u'accordion.gif\n', u'airplane.gif\n']


In [9]:
tl = ['f1.jpg','f2.gif', 'f3.png']
word = 'f2'
for tt in tl:
    if word == tt.split('.')[0]: print 'yeah, it is ', tt
    else: print 'not in the list'

not in the list
yeah, it is  f2.gif
not in the list


In [12]:
tl = ['f1.jpg','f2.gif', 'f3.png']
for tt in tl:
    tt += ': image!' # don't change tl
    print tt
print tl

f1.jpg: image!
f2.gif: image!
f3.png: image!
['f1.jpg', 'f2.gif', 'f3.png']


In [6]:
import re

idpv1 ="set up [phrasal verb] 1 : to prepare for something by putting things where they need to be We have one hour left to set up for the party. We had better start setting up. 2 set up (something) or set (something) up 2 a : to create or establish (something) for a particular purpose A panel needs to be set up to investigate the issue. She hopes that in a few years she can set up a private practice of her own. A foundation was set up for people with the disease. The church set up a school for the city's homeless children. The hikers set up camp [=they prepared an area where they could sleep outside] just before sunset. 2 b : to arrange and plan (an event or activity) I'll ask my secretary to set up a meeting with our clients. Let's set up a time to get together. He was the one who set up the bank robbery. 2 c : to put (something) in an upright position They set up flags along the street for the parade. Help me set the tent up. 2 d : to make (a machine, system, etc.) ready to be used They set up the movie camera next to the door. This computer wasn't set up to run so many programs at one time. 3 set (someone) up or set up (someone) 3 a : to cause (someone) to be in a bad situation or to appear guilty Those aren't his drugs. Someone must have set him up! [=framed him; made it look like the drugs were his] He claimed he was set up by the police. 3 b Brit : to help (someone) feel healthy and full of energy She says that daily exercise has set her up to face the challenges of getting old. 3 c : to give (someone) a job, a place to live, etc. She set me up in a great apartment in the city. He set his son up with a job at his company. 3 d : to do something that makes it likely or possible for (someone) to do, get, or experience something The team's excellent defense set them up to score the winning touchdown. usually + for I think you're just setting yourself up for a big disappointment. [=you're expecting something that won't happen and you will be disappointed when it doesn't] 3 e : to cause (someone) to meet someone in order to start a romantic relationship How did you first meet your husband? My best friend set us up. often + with She wants to set him up with her sister. 4 set (yourself) up as (something) 4 a : to try to make people believe that you are (something) She has set herself up as the best alternative to the current administration. He sets himself up as a defender of the people, but he's really only interested in getting more power. 4 b also set up as (something) or set up in (something) : to start your own business She set (herself) up as a wedding planner. [=she began working as a wedding planner] He set up in business after finishing college. see also SETUP"
idpv2 ="*idpv* take up [phrasal verb] 1 take up or take up (something) or take (something) up : to begin (something) again : to continue (something) after you or another person stops She took up [=picked up] the story where he left off. He is hoping that he and his ex-girlfriend can take up [=that they can continue their relationship] where they left off. 2 take up (something) : to fill (an area, amount of time, etc.) completely or almost completely The new couch takes up half of the room. I don't want to take up too much of your time, but I do have a few questions. The entire day was taken up by/with meetings. We don't use this table for anything. It's just taking up space. 3 take up (something) or take (something) up 3 a : to begin studying or practicing (an activity, subject, instrument, etc.) usually as a hobby I was thinking about taking up skiing/dancing/photography. She took up the guitar at age 11. taking up art lessons 3 b : to begin to deal with (a problem, an issue, etc.) The cause of global warming has been taken up by many celebrities recently. The court took up the question of how to deal with companies that break the law. He seemed willing to take up [=take on] the challenge. 3 c : to begin to have (a new job, home, etc.) He will take up his post [=begin working at his new post/job] at the beginning of the year. She went to France and took up residence in Paris. [=became a resident of Paris; began living in Paris] She took up [=she began to lead] the life of an artist. = She took up life as an artist. Two men with guns had taken up (their) positions on the roof. 3 d somewhat old-fashioned : to begin to use (something) They took up hammers and nails and went to work building the house. Once again, they took up arms [=picked up weapons and became ready to fight] to defend their country. 3 e : to make (something, such as a piece of clothing) shorter Can you take the legs of these pants up two inches? 3 f : to lift and remove (something) We took up the carpet in the living room and replaced it with hardwood flooring. 3 g : to gather (money, clothes, etc.) from many different people or places They are taking up a collection for the homeless shelter. 4 take (someone) up on (something) : to make an agreement with (someone) to accept (an offer) Can I buy you a drink? Sure, I'll take you up on that. We took the company up on its offer to replace the computer for free. 5 take (something) up with (someone) : to talk about (something, such as a problem) with (someone) If you have a problem, please take it up with one of our managers. Have you taken this up with your mother yet? 6 take up with (someone) : to begin a friendly or romantic relationship with (someone) After her divorce, she took up with a younger man. see also TAKE-UP"

idpv1 = re.sub(r'\s+(?=\d)', '\n', idpv1)
print idpv1
print '------'
tt2 = re.sub(r'\s+(?=\d)', '\n', idpv2)
print tt2

set up [phrasal verb]
1 : to prepare for something by putting things where they need to be We have one hour left to set up for the party. We had better start setting up.
2 set up (something) or set (something) up
2 a : to create or establish (something) for a particular purpose A panel needs to be set up to investigate the issue. She hopes that in a few years she can set up a private practice of her own. A foundation was set up for people with the disease. The church set up a school for the city's homeless children. The hikers set up camp [=they prepared an area where they could sleep outside] just before sunset.
2 b : to arrange and plan (an event or activity) I'll ask my secretary to set up a meeting with our clients. Let's set up a time to get together. He was the one who set up the bank robbery.
2 c : to put (something) in an upright position They set up flags along the street for the parade. Help me set the tent up.
2 d : to make (a machine, system, etc.) ready to be used They set